In [1]:
from encoder import *
import torchvision

In [2]:
config = SiglipVisionConfig(
    image_size=224,
    hidden_size=768,
    intermediate_size=3072,
    num_hidden_layers=12,
    num_attention_heads=12,
    num_channels=3,
    patch_size=16,
    layer_norm_eps=1e-6,
    attention_dropout=0.0,
    num_image_tokens=None
)

In [3]:
model = SiglipVisionModel(config)

In [5]:
img = torchvision.io.read_image("dog.jpg")
# resize image to 224x224
img = torchvision.transforms.functional.resize(img, (224, 224)).unsqueeze(0)
# normalize image
img = img / 255.0

In [6]:
output = model(img)
output.shape

torch.Size([1, 196, 768])

In [25]:
N, L, D = output.shape
keep_length = int(L * (1 - 0.75))
keep_length

49

In [36]:
noise = torch.rand(N, L)
ids_shuffle = torch.argsort(noise, dim=1)
ids_restore = torch.argsort(ids_shuffle, dim=1)

In [37]:
ids_keep.unsqueeze(-1).repeat(1, 1, D)

NameError: name 'ids_keep' is not defined

In [38]:
ids_keep = ids_shuffle[:, :keep_length]
# x_masked = torch.gather(output, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))

In [39]:
ids_keep

tensor([[ 54,  62, 169, 164,  40, 119, 103,  84, 175, 124,   7, 155,  30, 157,
         135, 172,  42,  44, 118, 192,  31,   6,  25,  64, 142, 116,  34,  82,
         147,  98, 109, 190,  95, 149, 140,   2,  65,  60,  66, 120,  78, 189,
          99, 117,  73,  49,  41,  14, 131]])